In [7]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="mesolitica/Speech-Translation-Instructions",
    repo_type='dataset',
    allow_patterns="data/*.parquet",
    local_dir="./Speech-Translation-Instructions",
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

'/home/mesolitica/stt/Speech-Translation-Instructions'

In [ ]:
from glob import glob
from tqdm import tqdm
from multiprocess import Pool
import itertools
import zipfile
import os

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)


def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

In [53]:
from glob import glob
import pandas as pd
import os
import json
from transformers import AutoProcessor

In [40]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
tokenizer = processor.tokenizer

In [10]:
files = sorted(glob('Speech-Translation-Instructions/data/*.parquet'))
files

['Speech-Translation-Instructions/data/longer-00000-of-00006.parquet',
 'Speech-Translation-Instructions/data/longer-00001-of-00006.parquet',
 'Speech-Translation-Instructions/data/longer-00002-of-00006.parquet',
 'Speech-Translation-Instructions/data/longer-00003-of-00006.parquet',
 'Speech-Translation-Instructions/data/longer-00004-of-00006.parquet',
 'Speech-Translation-Instructions/data/longer-00005-of-00006.parquet',
 'Speech-Translation-Instructions/data/train-00000-of-00024.parquet',
 'Speech-Translation-Instructions/data/train-00001-of-00024.parquet',
 'Speech-Translation-Instructions/data/train-00002-of-00024.parquet',
 'Speech-Translation-Instructions/data/train-00003-of-00024.parquet',
 'Speech-Translation-Instructions/data/train-00004-of-00024.parquet',
 'Speech-Translation-Instructions/data/train-00005-of-00024.parquet',
 'Speech-Translation-Instructions/data/train-00006-of-00024.parquet',
 'Speech-Translation-Instructions/data/train-00007-of-00024.parquet',
 'Speech-Trans

In [41]:
def loop(files):
    files, _ = files
    data = []
    for f in files:
        f_only = os.path.split(f)[1].replace('.parquet', '')
        df = pd.read_parquet(f)
        for i in tqdm(range(len(df))):
            new_f = os.path.join('Speech-Translation-Instructions-audio', 
                                 f'{f_only}-{i}.mp3')
            with open(new_f, 'wb') as fopen:
                fopen.write(df['audio_filename'].iloc[i]['bytes'])
            
            try:
                conversation = [
                    {"role": "user", "content": [
                        {"type": "audio", "audio_url": "audio.wav"},
                        {"type": "text", "text": df['question'].iloc[i]},
                    ]},
                    {"role": "assistant", "content": df['answer'].iloc[i]},
                ]
                text = processor.apply_chat_template(conversation, tokenize=False)
            except Exception as e:
                continue
            
            data.append({
                'text': text,
                'audio': new_f,
            })
    return data

In [42]:
processed = loop((files[:1], 0))

100%|█████████████████████████████████████████████████████████████████████████████████| 6317/6317 [00:02<00:00, 3063.06it/s]


In [43]:
len(processed)

6317

In [48]:
processed[-1]

{'text': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nAudio 1: <|audio_bos|><|AUDIO|><|audio_eos|>\nTranscribe the story in english.<|im_end|>\n<|im_start|>assistant\nThe phrase, often used in right-wing politics, gained popularity after the May 27 coup.<|im_end|>\n',
 'audio': 'Speech-Translation-Instructions-audio/longer-00000-of-00006-6316.mp3'}

In [49]:
import IPython.display as ipd
ipd.Audio(processed[-1]['audio']) 

In [50]:
processed = multiprocessing(files, loop, cores = min(len(files), 30))

100%|███████████████████████████████████████████████████████████████████████████████| 11427/11427 [00:04<00:00, 2344.73it/s]


In [51]:
len(processed)

312152

In [54]:
with open('prepare-Speech-Translation-Instructions.json', 'w') as fopen:
    json.dump(processed, fopen)